In [73]:
import os
import sys
import time
import copy

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torchvision import datasets

import PIL

import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

from livelossplot import PlotLosses

from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics import classification_report

from IPython.display import Image

from tqdm.notebook import tqdm



In [243]:
class RandomSphere(Dataset):
    
    """
    Class for generating dataset of a random
    sphere lying in a low dimensional manifold
    embedded in a higher dimensional space
    """
    
    def __init__(self, N=1000, n=100, k=3, r=10.0, D=50.0, max_norm=1000.0, mu=10, sigma=5):
        
        """
        :param N: total number of samples
        :type N: int
        :param k: low (k)-dimensional manifold 
        :type k: int
        :param n: dimension of manifold in which sphere is embedded
        :type n: int
        :param r: radius of the sphere
        :type r: float
        :param D: clamping limit for negative examples
        :type D: float
        :return: points
        :param max_norm: maximum possible norm that a point can have
        :type max_norm: float
        :param mu: mean of normal distribution from which we sample
        :type: float
        :param sigma: standard deviation of normal distribution from which we sample
        :type: float
        """
        
        self.N = N
        self.n = n
        self.k = k
        self.r = r
        self.D = D
        self.max_norm = max_norm
        self.mu = mu
        self.sigma = sigma
        
        self.x_ck = None
        """center of the sphere"""
        
        self.x_cn = None
        """center of sphere in higher dimension"""
        
        self.points_k = None
        """points sampled from the sphere in k-dim"""
        
        self.points_n = None
        """embedding of `self.points_k` in n-dim"""
        
        self.actual_distances = None
        """actual distance of points from the sphere's surface"""
        
        self.distances = None
        """clamped distance of the point from the sphere's surface"""
        
        
#         rdm_affine = torchvision.transforms.RandomAffine(degrees=0, translate=None)
#         rdm_affine_params = rdm_affine.get_params(degrees=(0, 0), translate=None, scale_ranges=None, shears=None, img_size=(1, self.n))
#         affine_lambda = lambda x: TF.affine(x, *rdm_affine_params)
        
#         self.affine_tfm = transforms.Compose([
#             transforms.ToPILImage(),
#             transforms.Lambda(random_affine_apply),
#             transforms.ToTensor()
#         ])
        """randomly sampled affine transform"""
        
        self.gen_center()
        self.gen_points()
        self.embed_in_n()
        self.compute_distances()
        
    def __len__(self):
        return self.points_n.shape[0]
    
    def __getitem__(self, idx):
        return self.points_n[idx], self.distances[idx]
        
        
    def gen_center(self):
        """generate a center in lower dimension"""
        self.x_ck = np.random.normal(self.mu, self.sigma, self.k)
        
    def gen_points(self):
        """generating points in k-dim and embedding in n-dim"""
        points_k = np.random.normal(self.mu, self.sigma, (self.N, self.k))
        points_k = points_k - self.x_ck
        
        norms = np.linalg.norm(points_k, axis=1, ord=2).reshape(-1, 1)
        points_k = (points_k / norms)
        
#         print(np.round(np.linalg.norm(points_k, axis=1, ord=2))[np.ceil(np.linalg.norm(points_k, axis=1, ord=2)) > 1])
        
        assert (np.round(np.linalg.norm(points_k, axis=1, ord=2)) == 1).all()
        
        points_k = self.r * points_k
        
        # constructing negative examples
        neg_norms = np.random.uniform(low=1 + np.finfo(np.float).eps,\
                                      high=self.max_norm, size=np.floor(self.N / 2).astype(np.int64))
        
        points_k[:np.floor(self.N / 2).astype(np.int64)] = (neg_norms.reshape(-1, 1) / self.r) * points_k[:np.floor(self.N / 2).astype(np.int64)]
        
        points_k = points_k + self.x_ck
        
        self.points_k = points_k
        
    
    def embed_in_n(self):
        
        # embedding the center
        self.x_cn = np.zeros(self.n)
        self.x_cn[:self.k] = self.x_ck
        tmp_x_cn = torch.from_numpy(self.x_cn.astype(np.float32))
        self.x_cn = self.affine_tfm(tmp_x_cn.reshape(1, -1)).reshape(-1)

        
        #embedding the points
        points_n = np.zeros((self.N, self.n))
        points_n[:, :self.k] = self.points_k
        tmp_points_n = torch.from_numpy(points_n.astype(np.float32))
        self.points_n = torch.Tensor([self.affine_tfm(tmp_points_n[i].reshape(1, self.n)).reshape(-1).tolist() for i in range(tmp_points_n.shape[0])])
        print(tmp_points_n.reshape(self.N, 1, self.n).shape, self.points_n.shape)
        print(np.round(np.linalg.norm(self.points_n[np.ceil(self.N / 2).astype(np.int64):] - self.x_cn, axis=1, ord=2))[np.round(np.linalg.norm(self.points_n[np.ceil(self.N / 2).astype(np.int64):] - self.x_cn, axis=1, ord=2)) != self.r])
#         assert (np.round(np.linalg.norm(self.points_n[np.ceil(self.N / 2).astype(np.int64):] - self.x_cn, axis=1, ord=2)) == self.r).all()
        
    def compute_distances(self):
        
        self.actual_distances = np.linalg.norm(self.points_n - self.x_cn, axis=1, ord=2) - self.r
        self.distances = np.clip(self.actual_distances, a_min=None, a_max=self.D)
        

    
    
    
    
    
    

In [244]:
sphere_dataset = RandomSphere()



torch.Size([1000, 1, 100]) torch.Size([1000, 100])
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.


In [245]:
sphere_dataset.x_cn

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [227]:
# np.linalg.norm(sphere_dataset.points_n[np.ceil(sphere_dataset.N / 2).astype(np.int64):] - sphere_dataset.x_cn, axis=1, ord=2)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [190]:
np.linalg.norm(sphere_dataset.points_k[np.ceil(sphere_dataset.N / 2).astype(np.int64):] - sphere_dataset.x_ck, axis=1, ord=2)




array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10

In [18]:
assert (np.ceil(np.linalg.norm(b, axis=1, ord=2)) == 1).all()

In [24]:
np.random.uniform(low=1 + np.finfo(np.float).eps, high=sys.maxsize, size=5)

array([8.09072177e+18, 3.57405711e+17, 2.08952825e+18, 6.92777335e+17,
       6.13746283e+18])

In [141]:
tfm = torchvision.transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomAffine(degrees=360, translate=(100, 100)),
    transforms.ToTensor()
])

ValueError: translation values should be between 0 and 1

In [121]:
p = np.random.randint(0, 10000)
p

9753

In [122]:
# tfm = torchvision.transforms.RandomAffine(degrees=360, translate=(1, 1))
a = np.random.normal(6, 3, size=(6, 3))

In [123]:
b = torch.Tensor(a)
b

tensor([[11.2969,  6.4757,  7.9529],
        [ 7.6221,  7.3124,  9.7036],
        [ 9.0756,  3.4229,  5.9954],
        [ 5.3699,  6.5220,  6.9025],
        [ 4.9396,  4.8198,  4.2098],
        [ 8.7355,  2.7360, -0.6074]])

In [130]:
tfm(b)

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [125]:
random_affine_params = tfm.transforms[1].get_params(degrees=(360, -360), translate=(0.2, 0.2), scale_ranges=None, shears=None, img_size=(1, 3))

random_affine_apply = lambda x : TF.affine(x, *random_affine_params)


tfm2 = torchvision.transforms.Compose([
    transforms.ToPILImage(),
    transforms.Lambda(random_affine_apply),
    transforms.ToTensor()
])

In [126]:
c = tfm2(b)
c.numpy()

array([[[0.        , 0.        , 0.        ],
        [0.        , 0.4       , 0.7254902 ],
        [0.        , 0.19215687, 0.8039216 ],
        [0.        , 0.8784314 , 0.49803922],
        [0.        , 0.972549  , 0.40784314],
        [0.        , 0.6666667 , 0.28235295]]], dtype=float32)

In [87]:
transforms.Lambda(random_affine_apply)

Lambda()

In [88]:
transforms.ToTensor()

ToTensor()

In [212]:
sphere_dataset.affine_tfm(torch.from_numpy(np.random.randn(1000, 100).astype(np.float32))).shape

torch.Size([1, 1000, 100])